<h3><b>CREATE DATA FRAMES.IPYNB</b><h3>
<p>Creates dataframes with all data and without excluded participants<p>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import jsonlines
from functools import reduce
import statistics
import math
import os
import json
import ast
import warnings
pd.options.mode.copy_on_write = True
pd.set_option('display.max_colwidth', None)
pd.set_option('future.no_silent_downcasting', True)
warnings.simplefilter(action='ignore', category=FutureWarning)

#uses various functions
from dataclean_func import *

In [21]:
#load in dataframes
    ##for example script: using a subset of pilot data to show how data cleaning works
df = pd.read_csv("csvs/blinded_data_full.csv", low_memory=False)
demographics=pd.read_csv("csvs/blinded_demography.csv", low_memory=False)
df.drop('Unnamed: 0', axis="columns", inplace=True)
demographics.drop('Unnamed: 0', axis="columns", inplace=True)
demographics.drop_duplicates(inplace=True)##remove duplicates

In [22]:
len(set(df.participant_no))

405

In [23]:
excluded_list=[] #create empty list of excluded participants

DEMOGRAPHICS

In [24]:
demographics[demographics.aligns!='True'].dropna() ##dropping those with missing data 

,prolific_age,prolific_sex,prolific_MH,time taken,self_report_gender,self_report_assigned-at-birth,self_report_age,aligns,participant_no
4,60,Male,Yes,66.133333,Male,Yes,61,False,5
11,31,Male,No,91.800000,Male,Yes,32,False,12
12,24,Male,No,86.350000,Male,Yes,20,False,13
34,54,Female,No,48.866667,Female,Yes,57,False,35
49,63,Male,No,55.300000,Male,Yes,64,False,50
85,20,Male,Yes,128.166667,Male,Yes,24,False,86
103,65,Male,Yes,51.133333,Male,Yes,60,False,104
124,73,Female,No,55.650000,Female,Yes,75,False,125
139,42,Male,Yes,55.450000,Male,Yes,43,False,140
141,37,Female,No,100.566667,Female,Yes,38,False,142


In [25]:
##will want to check these people individually
    #included if discrepancy is due to gender not matching what is assigned at birth
    #and if age is within 1 year
to_check=list(demographics[demographics.aligns!='True'].participant_no)
excluded_dem=[]
for participant_no in to_check:
    checking=demographics[demographics.participant_no==participant_no]
    age_discrepancy=int(checking.prolific_age)-int(checking.self_report_age)
    if -1 <= age_discrepancy <=1:
        ##check gender
        if checking.prolific_sex.iloc[0]!=checking.self_report_gender.iloc[0] and checking['self_report_assigned-at-birth'].iloc[0]=='Yes':
            excluded_list.append(participant_no)
            excluded_dem.append(participant_no)
    else:
        excluded_list.append(participant_no)
        excluded_dem.append(participant_no)
len(excluded_list)

10

Digit span and diagnosis

In [26]:
#digit span and diagnosis
digit_span=make_digit_span(df)
diagnosis=make_diagnosis(df)
excluded_digit_span=[]

##exclude those with digit span of 0 
for i in list(digit_span[digit_span.digit_span==0].participant_no):
    excluded_list.append(i)
    excluded_digit_span.append(i)

dem_df=(diagnosis.merge(digit_span, on='participant_no')).merge(demographics, on='participant_no')
len(excluded_list)

10

In [27]:
#check for those without automated diagnoses
diagnosis[~diagnosis['cleaned_diagnosis'].isin(['Yes', 'No'])]

,diagnosis,participant_no,cleaned_diagnosis,depression,anxiety,adhd,autism,ptsd,eating_disorder,ocd,bpd,bipolar,fnd,historical,unspecified


In [28]:
#manually check for those where cleaned diagnosis isn't what prolific says
dem_df[dem_df['prolific_MH'] != dem_df['cleaned_diagnosis']][['diagnosis', 'cleaned_diagnosis', 'participant_no', 'prolific_MH']]

,diagnosis,cleaned_diagnosis,participant_no,prolific_MH
14,no,No,15.0,Yes
25,no,No,26.0,Yes
28,"Yes, am currently having councelling. CBT",Yes,29.0,No
39,no,No,40.0,Yes
40,anxiety,Yes,41.0,No
48,PTSD Anxiety,Yes,49.0,No
69,no,No,70.0,Yes
74,depression,Yes,75.0,No
79,,No,80.0,Yes
80,depression,Yes,81.0,No


TIME LIMIT

In [29]:
##work out time taken from jatos -- as prolific is inaccurate
time_taken=[]
for participant in set(df.participant_no):
    sub_df=df[df.participant_no==participant]
    numbers=list(sub_df.time_elapsed)
    # Extract numbers that are greater than the next number
    peaks = [num for i, num in enumerate(numbers[:-1]) if num > numbers[i + 1]]
    peaks.append(numbers[-1])
    jatos_time_taken=np.sum(peaks)/60000 
    time_taken.append({
        'participant_no': participant,
        'jatos_time_taken': jatos_time_taken
    })
demographics=dem_df.merge(pd.DataFrame(time_taken), on='participant_no', how='left') ##CHANGED

In [30]:
demographics[demographics['jatos_time_taken']>120].sort_values(by='jatos_time_taken')

,diagnosis,participant_no,cleaned_diagnosis,depression,anxiety,adhd,autism,ptsd,eating_disorder,ocd,...,digit_span,prolific_age,prolific_sex,prolific_MH,time taken,self_report_gender,self_report_assigned-at-birth,self_report_age,aligns,jatos_time_taken
338,No,339.0,No,0,0,0,0,0,0,0,...,7.0,38,Male,No,120.700000,Male,Yes,38,True,120.435400
161,N/A,162.0,No,0,0,0,0,0,0,0,...,6.0,30,Female,No,121.133333,Female,Yes,30,True,120.714133
380,,381.0,No,0,0,0,0,0,0,0,...,7.0,66,Female,No,124.050000,Female,Yes,67,False,121.802700
259,,260.0,No,0,0,0,0,0,0,0,...,7.0,27,Male,No,124.416667,Male,Yes,27,True,123.363583
290,no,291.0,No,0,0,0,0,0,0,0,...,5.0,20,Male,No,126.216667,Male,Yes,21,False,125.471017
393,No,394.0,No,0,0,0,0,0,0,0,...,8.0,69,Male,Yes,126.266667,Male,Yes,68,False,126.065233
85,,86.0,No,0,0,0,0,0,0,0,...,8.0,20,Male,Yes,128.166667,Male,Yes,24,False,126.339900
8,,9.0,No,0,0,0,0,0,0,0,...,6.0,28,Male,No,128.033333,Male,Yes,28,True,126.626250
194,no,195.0,No,0,0,0,0,0,0,0,...,11.0,23,Female,No,128.000000,Female,Yes,23,True,127.422817
224,no,225.0,No,0,0,0,0,0,0,0,...,7.0,54,Female,Yes,128.333333,Female,Yes,54,True,127.705067


In [31]:
#exclude those who took too long
excluded_time=[]
for i in list(demographics[demographics['jatos_time_taken']>120].participant_no):
    excluded_list.append(i)
    excluded_time.append(i)
len(excluded_list)

26

VIDEO RATINGS

In [32]:
#create video ratings dataframes
total_participant=len(list(set(df.participant_no)))
ratings_df=pd.DataFrame()
chosen_stim_df=pd.DataFrame()
points_rating_df=pd.DataFrame()
for i in set(df.participant_no):     
    ##vid_ratings(df, i, "plot")
    vid_ratings_temp=vid_ratings(df, i, "rating_vids")
    ratings_df=pd.concat([ratings_df, vid_ratings_temp])
    chosen_stim_temp=vid_ratings(df, i, "chosen_stim")
    chosen_stim_df=pd.concat([chosen_stim_df, chosen_stim_temp])
    
    points_rating_temp=vid_ratings(df, i, "points_rating") 
    points_rating_df=pd.concat([points_rating_df, points_rating_temp])

chosen_stim_df=pd.concat([chosen_stim_df, points_rating_df])
chosen_stim_df=chosen_stim_df.fillna(0).sort_values(by='participant_no').reset_index().drop('index',axis=1)#

In [33]:
##exclude any with 0 for a rating of interest
excluded_disgust=[]
excluded_fear=[]
disgust_stim_check=chosen_stim_df[chosen_stim_df.trial_type=="disgust"]
fear_stim_check=chosen_stim_df[chosen_stim_df.trial_type=="fear"]
exclude=[]
for i in set(chosen_stim_df.participant_no):
    participant_disgust=disgust_stim_check[disgust_stim_check.participant_no==i].reset_index()
    if participant_disgust.disgusting_1[0]==0:
        exclude.append(i)
        excluded_disgust.append(i)

    participant_fear=fear_stim_check[fear_stim_check.participant_no==i].reset_index()
    if participant_fear.frightening_1[0]==0:
        exclude.append(i)
        excluded_fear.append(i)

#exclude ones with a zero
for i in list(exclude):
    excluded_list.append(i)
len(excluded_list)

43

In [34]:
filtered_disgust= chosen_stim_df[chosen_stim_df['participant_no'].isin(excluded_disgust)][['participant_no', 'trial_type', 'frightening_1', 'disgusting_1']]
filtered_disgust[filtered_disgust.trial_type=='disgust']

,participant_no,trial_type,frightening_1,disgusting_1
10,4.0,disgust,0.0,0.0
236,79.0,disgust,0.0,0.0


In [35]:
filtered_fear= chosen_stim_df[chosen_stim_df['participant_no'].isin(excluded_fear)][['participant_no', 'trial_type', 'frightening_1', 'disgusting_1', 'Vid']]
filtered_fear[filtered_fear.trial_type=='fear']

,participant_no,trial_type,frightening_1,disgusting_1,Vid
95,32.0,fear,0.0,0.0,0877
134,45.0,fear,0.0,0.0,0548
205,69.0,fear,0.0,0.0,0877
213,72.0,fear,0.0,0.0,0548
232,78.0,fear,0.0,0.0,0374
237,80.0,fear,0.0,0.0,0548
320,107.0,fear,0.0,0.0,0877
504,169.0,fear,0.0,0.0,0374
657,220.0,fear,0.0,0.0,0877
773,258.0,fear,0.0,0.0,0877


In [36]:
#combine with demographic information
chosen_stim_df=pd.merge(chosen_stim_df, dem_df, on='participant_no', how='outer')

BEHAVIOURAL TASK

In [ ]:
##create dataframe with all task information
complete_task_df=create_task_df(df, "no plot")

In [ ]:
#exclusions on the basis of this dataframe
task_understood=make_task_understood(df, complete_task_df, "no plot")
#task_understood[task_understood.task_understood=="No"]

for i in list(task_understood[task_understood.task_understood=="No"].participant_no):
    excluded_list.append(i)
excluded_list

In [ ]:
#extract task outcomes - i.e., error types, error rates, win-stay/lose-shift
task_summary=make_task_outcomes(complete_task_df.drop('index', axis=1))
task_summary

Add relevant video rating outcomes and demographic info to this 'task_summary' df - for hypothesis testing

In [ ]:
##adding video information to task dataframe (for hypothesis testing)
stim_ratings_covariates=pd.DataFrame()
block_feedback=pd.DataFrame()
for participant_no in set(chosen_stim_df.participant_no):
    participant_df=chosen_stim_df[chosen_stim_df.participant_no==participant_no]
    disgust=participant_df[participant_df.trial_type=="disgust"]
    fear=participant_df[participant_df.trial_type=="fear"]
    valence_diff=int(fear.unpleasant_1)-int(disgust.unpleasant_1)
    arousal_diff=int(fear.arousing_1)-int(disgust.arousing_1)

    valence_habdiff=(int(fear.unpleasant_1)-int(fear.unpleasant_2))-(int(disgust.unpleasant_1)-int(disgust.unpleasant_2))
    arousal_habdiff=(int(fear.arousing_1)-int(fear.arousing_2))-(int(disgust.arousing_1)-int(disgust.arousing_2))
    
    row=pd.DataFrame({
        'participant_no': [participant_no],
        'valence_diff': [valence_diff],
        'arousal_diff': [arousal_diff],
        'valence_habdiff': [valence_habdiff],
        'arousal_habdiff': [arousal_habdiff],
    })
    stim_ratings_covariates=pd.concat([stim_ratings_covariates, row])

    block_feedback_row=pd.DataFrame({
        'participant_no': [participant_no, participant_no, participant_no],
        'block_type': ['Fear', 'Disgust', 'Points'],
        'feedback_details': [fear['Vid'].iloc[-1], disgust['Vid'].iloc[-1], 'Lose 10 Points']
        #'disgust_vid': [disgust['Vid'].iloc[-1]],
        #'fear_vid': [fear['Vid'].iloc[-1]],
        #'points_feedback': 'Lose 10 Points'
    })
    block_feedback=pd.concat([block_feedback, block_feedback_row])

In [ ]:
##combine demographics, video ratings and task performance into one dataframe
task_and_ratings_summary=pd.merge(task_summary, stim_ratings_covariates, on='participant_no', how='outer')
task_and_ratings_summary=pd.merge(task_and_ratings_summary, block_feedback, on=['participant_no', 'block_type'], how='outer')
dfs = [task_and_ratings_summary, dem_df]
dem_vids_task = reduce(lambda  left,right: pd.merge(left,right,on=['participant_no'],
                                            how='outer'), dfs)

SAVE OUT as CSVs

In [ ]:
##remove excluded participants
if [] in list(excluded_list):
    excluded_list.remove([])
excluded_list=(set(excluded_list))
excluded_list

In [ ]:
##remove these participants and save out as csvs
excluded_chosen_stim=chosen_stim_df
excluded_ratings=ratings_df
excluded_complete_task=complete_task_df
excluded_demographics=demographics
excluded_dem_vids_task=dem_vids_task

for participant in excluded_list:
    excluded_chosen_stim=excluded_chosen_stim.loc[excluded_chosen_stim.participant_no!=participant]
    excluded_ratings=excluded_ratings.loc[excluded_ratings.participant_no!=participant]
    excluded_complete_task=excluded_complete_task.loc[excluded_complete_task.participant_no!=participant]
    excluded_demographics=excluded_demographics.loc[excluded_demographics.participant_no!=participant]
    excluded_dem_vids_task=excluded_dem_vids_task.loc[excluded_dem_vids_task.participant_no!=participant]

#excluded_chosen_stim.to_csv('csvs/chosen_stim_excluded.csv')
#excluded_ratings.to_csv('csvs/ratings_excluded.csv')
#excluded_complete_task.to_csv('csvs/complete_task_excluded.csv')
#excluded_demographics.to_csv('csvs/demographics_excluded.csv')
#excluded_dem_vids_task.to_csv('csvs/dem_vids_task_excluded.csv')

<p><b>GET DEMOGRAPHIC INFORMATION</b></p>
<p>e.g., for summary in results section</p>

In [ ]:
np.mean(excluded_demographics.prolific_age)

In [ ]:
excluded_demographics['prolific_sex'].value_counts(normalize=False)